# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,283.05,92,100,4.02,NZ,1720840808
1,1,tsiombe,-25.3000,45.4833,288.68,62,0,1.07,MG,1720840809
2,2,anadyr,64.7500,177.4833,290.43,51,40,7.00,RU,1720840811
3,3,tamanrasset,22.7850,5.5228,302.10,16,0,3.60,DZ,1720840813
4,4,edinburgh of the seven seas,-37.0676,-12.3116,285.36,68,99,15.85,SH,1720840814


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

import geoviews.tile_sources as gvts
import geoviews as gv

# Configure the map plot

# Define the dimensions for the data
humidity = gv.Dimension('Humidity')

# Create a GeoViews Dataset from the city_data_df DataFrame
city_points = gv.Points(city_data_df, ['Lng', 'Lat'], ['City', 'Country', 'Humidity'])

# Create the map with points where the size of the point is based on the humidity
map_plot = city_points.opts(
    size=gv.dim('Humidity')*0.1,
    color=gv.dim('City'),  # Map the 'City' dimension to the colors of the points
#    cmap='plasma', 
    tools=['hover'], 
  #  color=gv.dim('blue'),
    width=1000,
    height=600,
    cmap='tab20'
  
) * gv.tile_sources.OSM

# Display the map
map_plot

:Overlay
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity)
   .WMTS.I   :WMTS   [Longitude,Latitude]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter the city_data_df DataFrame
ideal_cities_df = city_data_df[
    ((city_data_df["Lat"] > 50) | (city_data_df["Lat"] < -35)) & 
    ((city_data_df["Lng"] < 20) | (city_data_df["Lng"] > 140)) &
    (((city_data_df["Max Temp"]-273.15)*9/5+32) < 85)
    ]

# Drop any rows with null values
ideal_cities_df.dropna(inplace=True)

# Display sample data
ideal_cities_df

C:\Users\johns\AppData\Local\Temp\ipykernel_30688\1367459158.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,283.05,92,100,4.02,NZ,1720840808
2,2,anadyr,64.7500,177.4833,290.43,51,40,7.00,RU,1720840811
4,4,edinburgh of the seven seas,-37.0676,-12.3116,285.36,68,99,15.85,SH,1720840814
7,7,mount gambier,-37.8333,140.7667,285.26,68,79,2.37,AU,1720840577
8,8,longyearbyen,78.2186,15.6401,277.06,93,75,3.60,SJ,1720840820
...,...,...,...,...,...,...,...,...,...,...
487,487,hertford,51.7959,-0.0785,286.51,91,100,3.53,GB,1720841580
501,501,fort mcmurray,56.7268,-111.3810,289.92,51,40,5.14,CA,1720841502
512,512,wynyard,-41.0000,145.7167,285.48,69,18,0.45,AU,1720841620
547,547,elverum,60.8819,11.5623,287.36,83,36,0.99,NO,1720841673


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,waitangi,NZ,-43.9535,-176.5597,92,
2,anadyr,RU,64.7500,177.4833,51,
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,68,
7,mount gambier,AU,-37.8333,140.7667,68,
8,longyearbyen,SJ,78.2186,15.6401,93,
...,...,...,...,...,...,...
487,hertford,GB,51.7959,-0.0785,91,
501,fort mcmurray,CA,56.7268,-111.3810,51,
512,wynyard,AU,-41.0000,145.7167,69,
547,elverum,NO,60.8819,11.5623,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Limit the search to return only the nearest hotel
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    #params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(params)

    # Convert the API response to JSON format
    name_address = name_address.json() 

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'categories': 'accommodation.hotel', 'limit': 1, 'apiKey': '21cf05dbe0ac4d03baf225add1dd5595', 'filter': 'circle:-176.5597,-43.9535,10000'}
waitangi - nearest hotel: Hotel Chathams
{'categories': 'accommodation.hotel', 'limit': 1, 'apiKey': '21cf05dbe0ac4d03baf225add1dd5595', 'filter': 'circle:177.4833,64.75,10000'}
anadyr - nearest hotel: Гостевой дом
{'categories': 'accommodation.hotel', 'limit': 1, 'apiKey': '21cf05dbe0ac4d03baf225add1dd5595', 'filter': 'circle:-12.3116,-37.0676,10000'}
edinburgh of the seven seas - nearest hotel: No hotel found
{'categories': 'accommodation.hotel', 'limit': 1, 'apiKey': '21cf05dbe0ac4d03baf225add1dd5595', 'filter': 'circle:140.7667,-37.8333,10000'}
mount gambier - nearest hotel: Commodore on the Park
{'categories': 'accommodation.hotel', 'limit': 1, 'apiKey': '21cf05dbe0ac4d03baf225add1dd5595', 'filter': 'circle:15.6401,78.2186,10000'}
longyearbyen - nearest hotel: Funken Lodge (formerly Spitsbergen Hotel)
{'categories': 'acc

,City,Country,Lat,Lng,Humidity,Hotel Name
0,waitangi,NZ,-43.9535,-176.5597,92,Hotel Chathams
2,anadyr,RU,64.7500,177.4833,51,Гостевой дом
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,68,No hotel found
7,mount gambier,AU,-37.8333,140.7667,68,Commodore on the Park
8,longyearbyen,SJ,78.2186,15.6401,93,Funken Lodge (formerly Spitsbergen Hotel)
...,...,...,...,...,...,...
487,hertford,GB,51.7959,-0.0785,91,The White Horse
501,fort mcmurray,CA,56.7268,-111.3810,51,Clearwater Suite Hotel
512,wynyard,AU,-41.0000,145.7167,69,Hotel Federal
547,elverum,NO,60.8819,11.5623,83,Thon Partner Hotel Central


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Create a GeoViews Dataset from the city_data_df DataFrame
hotel_points = gv.Points(hotel_df, ['Lng', 'Lat'], ['City', 'Country', 'Humidity', 'Hotel Name'])

# Create the map with points where the size of the point is based on the humidity
map_plot = hotel_points.opts(
    size=gv.dim('Humidity')*0.1,
    color=gv.dim('City'),  # Map the 'City' dimension to the colors of the points
#    cmap='plasma', 
    tools=['hover'], 
  #  color=gv.dim('blue'),
    width=1000,
    height=600,
    cmap='tab20'
  
) * gv.tile_sources.OSM

# Display the map
map_plot

:Overlay
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)
   .WMTS.I   :WMTS   [Longitude,Latitude]